In [1]:
import sys
sys.path.append("../")
import numpy as np
import pandas as pd
import os
import cv2

from dataset.aptos import APTOS

import torch
from torch.utils.data import Dataset, DataLoader

# Which GPU to use
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

In [2]:
aptos_train = APTOS('/ssd_Samsung870_2T/hc701-fed/hc701-project')
aptos_train_loader = DataLoader(aptos_train, batch_size=4, shuffle=True, num_workers=0)

### DensnNet121

In [3]:
# DenseNet121 baseline
from model.baseline import Baseline
DesenNet121 = Baseline('densenet121', 5, pretrained=False)
Loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(DesenNet121.parameters(), lr=0.001)

In [4]:
# Train
DesenNet121.to(device)
DesenNet121.train()
for epoch in range(10):
    for i, (image, label) in enumerate(aptos_train_loader):
        image = image.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = DesenNet121(image)
        loss = Loss(output, label)
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print('Epoch: {}, Iteration: {}, Loss: {}'.format(epoch, i, loss.item()))

Epoch: 0, Iteration: 0, Loss: 1.7756930589675903
